Desarrollo API: Propones disponibilizar los datos de la empresa usando el framework FastAPI. Las consultas que propones son las siguientes:

* Película con mayor duración con filtros opcionales de AÑO, PLATAFORMA Y TIPO DE DURACIÓN. (la función debe llamarse get_max_duration(year, platform, duration_type))
* Cantidad de películas por plataforma con un puntaje mayor a XX en determinado año (la función debe llamarse get_score_count(platform, scored, year))
* Cantidad de películas por plataforma con filtro de PLATAFORMA. (La función debe llamarse get_count_platform(platform))
* Actor que más se repite según plataforma y año. (La función debe llamarse get_actor(platform, year))

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Importacion de librerías
import pandas as pd
import numpy as np
# Gestionar las rutas
import utils.paths as path

In [3]:
dataset_dir = path.make_dir_function("dataset")
api_data_dir = dataset_dir("api_data.parquet")

In [4]:
api_data_dir

PosixPath('/mnt/d/Backup/GitHub/pi_mlops/dataset/api_data.parquet')

In [4]:
# Carga del dataset del API
movies_scores_df = pd.read_parquet(api_data_dir)

In [5]:
# Veamos que columnas tenemos
movies_scores_df.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration_int,duration_type,listed_in,description,score
0,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113.0,min,"comedy, drama",a small fishing village must procure a local d...,3.5
1,as2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,110.0,min,"drama, international",a metro family decides to fight a cyber crimin...,3.5


<p align="justify">Película con mayor duración con filtros opcionales de AÑO, PLATAFORMA Y TIPO DE DURACIÓN. (la función debe llamarse get_max_duration(year, platform, duration_type))</p>


In [6]:
def get_max_duration(year: int,platform: str, duration_type: str):
    mask_year = movies_scores_df["release_year"] == year
    mask_platform = movies_scores_df['show_id'].str[0] == platform[0]
    mask_duration_type = movies_scores_df['duration_type'] == duration_type
    mask_movie_type = movies_scores_df['type'] == 'movie'
    result = movies_scores_df[mask_year&mask_platform&mask_duration_type&mask_movie_type].sort_values(by='duration_int', ascending=False).iloc[0]
    return {
        'title': result.title,
        'platform': platform,
        'duration_type': duration_type,
        'duration_int': int(result.duration_int),
        'year': year,
    }

In [7]:
# 
get_max_duration(2010,'hulu','min')

{'title': 'how do you know',
 'platform': 'hulu',
 'duration_type': 'min',
 'duration_int': 121,
 'year': 2010}

Cantidad de películas por plataforma con un puntaje mayor a XX en determinado año (la función debe llamarse get_score_count(platform, scored, year))

In [8]:

def get_score_count(platform: str, score: float, year: int):
    mask_movie_type = movies_scores_df['type'] == 'movie'
    mask_platform = movies_scores_df['show_id'].str[0] == platform[0]
    mask_more_than_score = movies_scores_df['score'] > score
    mask_year = movies_scores_df['release_year'] == year
    count = movies_scores_df[mask_movie_type&mask_platform&mask_more_than_score&mask_year].shape[0]
    return {
        'platform': platform, 
        'score': score,
        'year': year,
        'count': count
    }

In [9]:
# Cantidad de películas que tieien un score mayor a 0.5 que fueron lanzadas en el 2010 en la plataforma netflix
get_score_count('netflix',0.5,2010)

{'platform': 'netflix', 'score': 0.5, 'year': 2010, 'count': 154}

Cantidad de películas por plataforma con filtro de PLATAFORMA. (La función debe llamarse get_count_platform(platform))

In [10]:
def get_count_platform(platform: str):
    mask_platform = movies_scores_df['show_id'].str[0] == platform[0]
    count = movies_scores_df[mask_platform].shape[0]
    return {
        "platform": platform,
        "count": count
    }

In [11]:
# Cantidad de películas en la plataforma amazon
get_count_platform('hulu')

{'platform': 'hulu', 'count': 3073}

Actor que más se repite según plataforma y año. (La función debe llamarse get_actor(platform, year))

In [75]:
from itertools import chain
from collections import Counter

def get_actor(platform: str, year: int, none_val=""):
    
    mask_platform = movies_scores_df['show_id'].str[0] == platform[0]
    mask_year = movies_scores_df["release_year"] == year
    cast_list = movies_scores_df[mask_platform&mask_year]['cast'].apply(lambda x: str(x).split(', '))
    actors = list(chain.from_iterable(cast_list))
    actors = list(map(str.strip, actors))
    if none_val == "":    
        actor_counts = Counter(actors)
        most_common_actor, count = actor_counts.most_common(1)[0]
        return {
            "platform": platform,
            "most_common_actor": most_common_actor,
            "count": count
        }
    elif none_val == "not_none":
        actors = list(filter(lambda x: x.lower() != "none", actors))
        if len(actors) > 0:
            actor_counts = Counter(actors)
            most_common_actor, count = actor_counts.most_common(1)[0]
            return {
                "platform": platform,
                "most_common_actor": most_common_actor,
                "count": count
            }
        else:
            return {
                "platform": platform,
                "most_common_actor": "No existen valores no nulos",
                "count": 0
            }

In [76]:
# Actor más popular (que más se repite) en la plataforma amazon durante el año 2010
get_actor("hulu",2019, "")

{'platform': 'hulu', 'most_common_actor': 'None', 'count': 412}

In [77]:
get_actor("hulu",2019, "not_none")

{'platform': 'hulu',
 'most_common_actor': 'No existen valores no nulos',
 'count': 0}